<a href="https://colab.research.google.com/github/whereissam/gemini-workshop/blob/main/Part_1_text_prompting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2025 Patrick Loeber

In [ ]:

#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Workshop: Build with Gemini (Part 1)

<a target="_blank" href="https://colab.research.google.com/github/patrickloeber/workshop-build-with-gemini/blob/main/notebooks/part-1-text-prompting.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This workshop teaches how to build with Gemini using the Gemini API and Python SDK.

Course outline:

- **Part1 (this notebook): Quickstart + Text prompting**
  - Text understanding
  - Streaming response
  - Chats
  - System prompts
  - Config options
  - Long context
  - Token usage
  - Final excercise: Chat with book

- **[Part 2: Multimodal understanding (image, video, audio, docs, code)](https://github.com/patrickloeber/workshop-build-with-gemini/blob/main/notebooks/part-2-multimodal-understanding.ipynb)**

- **[Part 3: Thinking models + agentic capabilities (tool usage)](https://github.com/patrickloeber/workshop-build-with-gemini/blob/main/notebooks/part-3-thinking-and-tools.ipynb)**

## 0. Use the Google AI Studio as playground

Explore and play with all models in the [Google AI Studio](https://aistudio.google.com/apikey).


## 1. Setup


Get a free API key in the [Google AI Studio](https://aistudio.google.com/apikey)

In [ ]:
from google.colab import userdata

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

Install the [Google Gen AI Python SDK](https://github.com/googleapis/python-genai)

In [ ]:
%pip install -q -U google-genai

Configure Client

In [ ]:
from google import genai
from google.genai import types

client = genai.Client(api_key=GOOGLE_API_KEY)

Configure model. See all [models](https://ai.google.dev/gemini-api/docs/models)

In [ ]:
MODEL = "gemini-2.0-flash" # TODO

## 2. Send your first prompt

In [ ]:
response = client.models.generate_content(
    model=MODEL,
    contents="Explain how AI works",
)

print(response.text)

Okay, let's break down how Artificial Intelligence (AI) works in a way that's understandable.  Think of it like this: AI isn't a single magical thing, but rather a collection of techniques and approaches that allow computers to perform tasks that typically require human intelligence.

**The Core Idea: Learning from Data**

At its heart, most AI relies on the idea of **learning from data**.  Instead of being explicitly programmed with specific instructions for *every* situation, AI systems are given vast amounts of data and algorithms that enable them to:

1.  **Identify Patterns:**  The AI looks for patterns, correlations, and relationships within the data.
2.  **Build a Model:**  Based on the identified patterns, the AI creates a model (a representation) of the underlying relationships. This model could be a set of rules, a complex mathematical equation, or a neural network.
3.  **Make Predictions or Decisions:**  When new data is presented, the AI uses its model to make predictions, 

#### **!! Exercise !!**
- Send a few more prompts
  - Tell Gemini to write a blog post about the transformers architecture
  - Ask Gemini to explain list comprehension in Python
- Experiment with models:
  - Try Gemini 2.0 Flash-Lite
  - Try Gemini 2.5 Pro Exp

In [ ]:
# TODO

## 3. Text understanding

The simplest way to generate text is to provide the model with a text-only prompt. `contents` can be a single prompt, a list of prompts, or a combination of multimodal inputs.

In [ ]:
response = client.models.generate_content(
    model=MODEL,
    contents="Explain how AI works, history",
)

print(response.text)

#### Streaming response

By default, the model returns a response after completing the entire text generation process. You can achieve faster interactions by using streaming to return outputs as they're generated.

In [ ]:
response = client.models.generate_content_stream(
    model=MODEL,
    contents="Explain how AI works, AI Model: openai",
)

for chunk in response:
    print(chunk.text)
    print("_" * 80)

Let
________________________________________________________________________________
's break down how AI works, particularly focusing on the principles behind AI models
________________________________________________________________________________
 like those developed by OpenAI.  It's a vast topic, so we'll hit
________________________________________________________________________________
 the key concepts and then drill down a bit on the specific techniques OpenAI often employs.

**The Core Idea: Learning from Data**

At its heart,
________________________________________________________________________________
 AI, especially the kind that powers ChatGPT and similar systems, is about enabling computers to *learn from data* without being explicitly programmed for every possible scenario.  Instead of rigid
________________________________________________________________________________
, step-by-step instructions, AI systems are trained on massive datasets, allowing them to ident

#### Chat

The SDK chat class provides an interface to keep track of conversation history. Behind the scenes it uses the same `generate_content` method.

In [ ]:
chat = client.chats.create(
    model="gemini-2.0-flash",
    history=[
        types.Content(role="user", parts=[types.Part(text="Hello")]),
        types.Content(
            role="model",
            parts=[
                types.Part(
                    text="Great to meet you. What would you like to know?"
                )
            ],
        ),
    ],
)

response = chat.send_message_stream(message="I have 2 dogs in my house.")
for chunk in response:
    print(chunk.text)
    print("_" * 80)

That
________________________________________________________________________________
's wonderful
________________________________________________________________________________
! Dogs are great companions. Tell me a little more about them! What kind of dogs are
________________________________________________________________________________
 they? What are their names? How old are they? I'd love to
________________________________________________________________________________
 hear about your furry friends!

________________________________________________________________________________


In [ ]:
response = chat.send_message_stream(message="How many paws are in my house?")
for chunk in response:
    print(chunk.text)
    print("_" * 80)

print(chat.get_history())

Since
________________________________________________________________________________
 you have two
________________________________________________________________________________
 dogs, and each dog has four paws, there are a total of 8
________________________________________________________________________________
 paws in your house (2 dogs * 4 paws/dog = 8
________________________________________________________________________________
 paws).

________________________________________________________________________________
[Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text='Hello')], role='user'), Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text='Great to meet you. What would you like to know?')], role='mo

#### Parameters

Every prompt you send to the model includes parameters that control how the model generates responses. You can configure these parameters, or let the model use the default options.

In [ ]:
# TODO

- `max_output_tokens`: Sets the maximum number of tokens to include in a candidate.
- `temperature`: Controls the randomness of the output. Use higher values for more creative responses, and lower values for more deterministic responses. Values can range from [0.0, 2.0].
- `top_p`: Changes how the model selects tokens for output. Tokens are selected from the most to least probable until the sum of their probabilities equals the top_p value.
- `top_k`: Changes how the model selects tokens for output. A top_k of 1 means the selected token is the most probable among all the tokens in the model's vocabulary, while a top_k of 3 means that the next token is selected from among the 3 most probable using the temperature. Tokens are further filtered based on top_p with the final token selected using temperature sampling.
- `stop_sequences`: List of strings  (up to 5) that tells the model to stop generating text if one of the strings is encountered in the response. If specified, the API will stop at the first appearance of a stop sequence.
- `seed`: If specified, the model makes a best effort to provide the same response for repeated requests. By default, a random number is used.

#### System instructions

System instructions let you steer the behavior of a model based on your specific use case. When you provide system instructions, you give the model additional context to help it understand the task and generate more customized responses. The model should adhere to the system instructions over the full interaction with the user, enabling you to specify product-level behavior separate from the prompts provided by end users.

In [ ]:
# TODO

#### Long context and token counting

Gemini 2.0 Flash and 2.5 Pro have a 1M token context window.

In practice, 1 million tokens could look like:

- 50,000 lines of code (with the standard 80 characters per line)
- All the text messages you have sent in the last 5 years
- 8 average length English novels
- 1 hour of video data

Let's feed in an entire book and ask questions:



In [ ]:
import requests
res = requests.get("https://gutenberg.org/cache/epub/16317/pg16317.txt")
book = res.text

In [ ]:
print(book[:100])

﻿The Project Gutenberg eBook of The Art of Public Speaking
    
This ebook is for the use of anyon


In [ ]:
print(f"# charakters {len(book)}")
print(f"# words {len(book.split())}")
print(f"# tokens: ~{int(len(book.split()) * 4/3)}")   # rule of thumb: 100tokens=75words

# charakters 979714
# words 162461
# tokens: ~216614


In [ ]:
prompt = f"""
Summerize the book, REturn 10 bullet points

{book}
"""

response = client.models.generate_content(
    model=MODEL,
    contents=prompt
)

print(response.text)

Okay, here is a 10-bullet-point summary of "The Art of Public Speaking" by J. Berg Esenwein and Dale Carnegie:

*   **Overcome Fear with Practice and Subject Absorption:** Public speaking confidence comes from facing audiences frequently, not just reading about it. Focusing on your subject matter and knowing it well helps to minimize self-consciousness.
*   **Avoid Monotony Through Vocal Variation:** Monotony is deadly to a speech. Conquer it by mastering the principles of emphasis, pitch, pace, and pause to create a dynamic and engaging delivery.
*   **Master Emphasis and Subordination:** Emphasize important words through contrast (volume, pace, pitch) while subordinating less important words to guide the audience's attention effectively.
*   **Vary Pitch to Engage the Audience:** Use continual changes in vocal pitch, from high to low, to add expression and prevent monotony. Different thoughts and emotions call for different pitches.
*   **Change Pace for Impact:** Vary the tempo (spe

To understand the token usage, you can check


`usage_metadata`:

In [ ]:
print(response.usage_metadata.cached_content_token_count)
print(response.usage_metadata.candidates_token_count)

None
441


You can also use `count_tokens` to check the size of your input prompt(s):

In [ ]:
client.models.count_tokens(model=MODEL, contents=prompt)

CountTokensResponse(total_tokens=250554, cached_content_token_count=None)

## !! Exercise: Chat with a book !!

Task:
- Create a chat
- Use a system prompt: `"You are an expert book reviewer with a witty tone."`
- Use a temperature of `1.5`
- Ask 1 to summarize the book
- Ask 1 question to explain more detail about a certain topic from the book
- Ask to create a social media post based on the book
- Print the total number of tokens used during the chat

In [ ]:
# TODO

## Recap & Next steps

Nice work! You learned
- Python SDK quickstart
- Text prompting
- Streaming and chats
- System prompts and config options
- Long context and token counting


More helpful resources:
- [API docs quickstart](https://ai.google.dev/gemini-api/docs/quickstart?lang=python)
- [Text generation docs](https://ai.google.dev/gemini-api/docs/text-generation)
- [Long context docs](https://ai.google.dev/gemini-api/docs/long-context)

Next steps:
- [Part 2: Multimodal understanding (image, video, audio, docs, code)](https://github.com/patrickloeber/workshop-build-with-gemini/blob/main/notebooks/part-2-multimodal-understanding.ipynb)